In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
import geopandas as gpd
import rtree
import fiona.crs
import pyproj
import shapely.geometry as geom
import csv


In [7]:
proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
zones = gpd.read_file('dataset_mh5172/500cities_tracts.geojson')
zones = zones.to_crs({'init' :'epsg:3857'})

In [13]:
# zones = gpd.read_file('dataset_mh5172/500cities_tracts.geojson')
point = geom.Point(-78.58814956,35.82840584)

In [14]:
def createIndex(shapefile):# target geojson
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile)
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findZone(p, index, zones):
    Point = index.intersection((p.x, p.y, p.x, p.y))
    for idx in Point:
        if zones.geometry[idx].contains(p):
            return idx
    return None

index, zones = createIndex('dataset_mh5172/500cities_tracts.geojson')
result = findZone(point, index, zones)
result

21313

In [17]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
#this function is copied from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

def createIndex(shapefile):# target geojson
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile)
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findZone(p, index, zones):
    Point = index.intersection((p.x, p.y, p.x, p.y))
    for idx in Point:
        if zones.geometry[idx].contains(p):
            return idx
    return None

def DetectDrug(pid,Twitter):
    import csv
    import pyproj
    import shapely.geometry as geom
#     proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
    index, zones = createIndex('dataset_mh5172/500cities_tracts.geojson')
    DrugKeyWord = []
    with open('dataset_mh5172/drug_illegal.txt') as file:
        reader = csv.reader(file)
        for word in reader:
            DrugKeyWord.append(word)
    with open('dataset_mh5172/drug_sched2.txt') as file:
        reader = csv.reader(file)
        for word in reader:
            DrugKeyWord.append(word)
    reader = csv.reader(Twitter,delimiter='|')
    for row in reader:
        TwitterContent = deEmojify(row[6])
        for i in row[6].split(' '):
            if i not in DrugKeyWord:
                Lon, Lat = row[1], row[2]
                Location = geom.Point(PickupLon, PickupLat)
                census = findZone(Location, index, zones)
                yield (census,1)
                break

In [19]:
rdd = sc.textFile('dataset_mh5172/tweets.csv')
counts = rdd.mapPartitionsWithIndex(DetectDrug).reduceByKey(lambda x,y:x+y).take(100)
counts

KeyboardInterrupt: 